In [140]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np



url="https://www.leboncoin.fr/voitures/offres/ile_de_france/?th=1&q=renault%20zoe"

class Annonce:
    def __init__(self, idAnnonce, version, annee, kilometrage, prix, telephone, isPro, description):
        self.idAnnonce = idAnnonce
        self.version = version
        self.annee = annee
        self.kilometrage = kilometrage
        self.prix = prix
        self.telephone = telephone
        self.isPro = isPro
        self.description = description
    
    def to_dict(self):
        return {
            'idAnnonce': self.idAnnonce,
            'annee': self.annee,
            'kilometrage':self.kilometrage, 
            'prix':self.prix,
            'description':self.description
        }

def dicoFeatures(x):
    return {
        'a': 1,
        'b': 2
    }.get(x, 9)
    
def getSoupFromURL(url, method='get', data={}):

    if method == 'get':
        res = requests.get(url)
    elif method == 'post':
        res = requests.post(url, data=data)
    else:
        return None

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        return soup
    else:
        return None

def getDetail(urlItem, cpt):
    #print (a2)
    annonce = Annonce(0, 0,0,0,0,0,0,0)
    if urlItem != None:
        soupItem = getSoupFromURL(urlItem)
        if soupItem:
            b = soupItem.find_all(class_="line")
            for b0 in b:
                #print (b0)
                b1 = b0.find_all(class_="property")
                b1Prime = b0.find_all(class_="value")
                                                   
                #print ("++++")
                for b2 in b1:
                    for  b2Prime in b1Prime:
                        annonce.idAnnonce = cpt
                        #print ((b2.text).upper())
                        if (b2.text).upper() == "PRIX":
                            annonce.prix = b2Prime.text.strip()
                            #print(b2Prime.text)
                        if (b2.text).upper() == "ANNÉE-MODÈLE":
                            annonce.annee = b2Prime.text.strip()
                        if (b2.text).upper() == "KILOMÉTRAGE":
                            annonce.kilometrage = b2Prime.text.strip()
                        if (b2.text).upper() == "DESCRIPTION :":
                            annonce.description = b2Prime.text
                        
                        #df[b2] = b2Prime                        
                        #print (b2Prime.text)
                        #df[b2.text]=b2Prime.text
                        
    return annonce

                    
def getInfoVoitures(url):
    list = []
    columns = ["version","annee","kilometrage","Prix","telephone","isPro"]

    #dummyarray = np.empty((6,6))
    #dummyarray[:] = np.nan

    df = pd.DataFrame(columns=columns)

    soup = getSoupFromURL(url)
    if soup:
        #a = soup.find_all(class_="item_infos")
        a = soup.find_all(class_="tabsContent")
        for  a0 in a:
            a1 = a0.find_all("li")
            cpt=0
            for a2 in a1:
                urlItem = "http:" + a2.find ("a")["href"]
                
                #print ("-------"+str(cpt))
                annonce = getDetail(urlItem, cpt)  
                
                #print (":::::")
                #print (annonce.idAnnonce)
                #print (":::::")
                
                #print(a2.find(class_="item_title").text)
                if a2.find(class_="ispro")!= None:
                    #print(a2.find(class_="ispro").text)
                    annonce.isPro = a2.find(class_="ispro").text
                else:
                    #print ('(NonPro)')
                    annonce.isPro = "(NonPro)"
                #print(a2.find(class_="item_price").text)
                cpt=cpt+1
                #print (annonce.annee)
                list.append(annonce)
            
    else:
        0
    return list

list = getInfoVoitures(url)

df = pd.DataFrame.from_records([s.to_dict() for s in list])
df.to_csv(r'pandas.csv', header=None, index=None, sep=',', mode='a')
